<a href="https://colab.research.google.com/github/ranjith13119/Chatbot-Attention-Seq2Seq.ipynb/blob/main/Chatbot_Attention_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
lines = open('/content/drive/MyDrive/NLP/Chatbot/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('/content/drive/MyDrive/NLP/Chatbot/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [3]:
exchn = []
for conver in conversations:
  exchn.append(conver.split(' +++$+++ ')[-1][1:-1].replace("'", " ").replace(",","").split())

In [4]:
diag = {}
for line in lines:
    diag[line.split(' +++$+++ ')[0]] = line.split(' +++$+++ ')[-1]

In [5]:
questions = [] # ---> Input
answers = []  # --> Target

In [6]:
for conver in exchn:
    for i in range(len(conver) - 1):
        questions.append(diag[conver[i]]) # THe first one is the Question id
        answers.append(diag[conver[i+1]]) # the next line item is the answer id

In [7]:
sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 20:  # taking small 
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])

In [8]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

In [9]:
clean_ques = []
clean_ans = []

In [10]:
import re
for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))

In [11]:
for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:20])

In [12]:
clean_ans=clean_ans[:30000]
clean_ques=clean_ques[:30000]

In [13]:
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1 
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [14]:
# remove the word which is having less than 10% occurances
thresh = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1        

In [15]:
for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'

In [16]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

In [17]:
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [18]:
### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}

In [19]:
# check whether the word is precent in the vocab. Else assign the word as <OUT>
encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

In [20]:
decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 20, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 20, padding='post', truncating='post')

In [22]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 20, padding='post', truncating='post')

In [23]:
VOCAB_SIZE = len(vocab)
MAX_LEN = 20

print(decoder_final_output.shape, decoder_inp.shape, encoder_inp.shape, len(vocab), len(inv_vocab), inv_vocab[0])

(30000, 20) (30000, 20) (30000, 20) 3734 3733 <PAD>


In [24]:
decoder_final_output

array([[  22,  357,    5, ...,  549,   13,    0],
       [3731, 1782,    4, ...,   46,    4,    0],
       [  42, 3730,    0, ...,    0,    0,    0],
       ...,
       [ 147,  375, 1608, ...,    0,    0,    0],
       [3730,    0,    0, ...,    0,    0,    0],
       [3730,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [25]:
from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [26]:
decoder_final_output.shape

(30000, 20, 3734)

In [27]:
embeddings_index = {}
with open('/content/drive/MyDrive/NLP/unsupervised Text summary/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

Glove Loded!


In [28]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = embedding_matrix_creater(50, word_index=vocab)

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate, Dropout, Attention

In [30]:
embed = Embedding(VOCAB_SIZE+1, 
                  50,                   
                  input_length=20,
                  trainable=True)

embed.build((None,))
embed.set_weights([embedding_matrix])

In [31]:
enc_inp = Input(shape=(20, ))

In [32]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
      This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
      There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True) # for decoder stats
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)  # for encoder output
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)  # fully connected layer

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
          inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """              
              Step function for computing energy for a single decoder state
              inputs: (batchsize * 1 * de_in_dim)
              states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [33]:
enc_embed = embed(enc_inp)
enc_lstm = Bidirectional(LSTM(400, return_state=True, dropout=0.05, return_sequences = True))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = enc_lstm(enc_embed)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

enc_states = [state_h, state_c]

dec_inp = Input(shape=(20, ))
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400*2, return_state=True, return_sequences=True, dropout=0.05)
output, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

# attention
attn_layer = AttentionLayer()
attn_op, attn_state = attn_layer([encoder_outputs, output])
decoder_concat_input = Concatenate(axis=-1)([output, attn_op])


dec_dense = Dense(VOCAB_SIZE, activation='softmax')
final_output = dec_dense(decoder_concat_input)

model = Model([enc_inp, dec_inp], final_output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 50)       186750      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 20, 800), (N 1443200     embedding[0][0]              

In [34]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [35]:
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=10, batch_size=24, validation_split=0.15) # with minimal 

Epoch 1/10
1063/1063 [==============================] - 169s 148ms/step - loss: 2.4287 - acc: 0.5983 - val_loss: 2.1187 - val_acc: 0.6367
Epoch 2/10
1063/1063 [==============================] - 155s 146ms/step - loss: 2.1185 - acc: 0.6220 - val_loss: 2.0439 - val_acc: 0.6439
Epoch 3/10
1063/1063 [==============================] - 155s 146ms/step - loss: 1.9877 - acc: 0.6293 - val_loss: 2.0269 - val_acc: 0.6451
Epoch 4/10
1063/1063 [==============================] - 157s 147ms/step - loss: 1.8546 - acc: 0.6358 - val_loss: 2.0404 - val_acc: 0.6482
Epoch 5/10
1063/1063 [==============================] - 157s 148ms/step - loss: 1.6953 - acc: 0.6473 - val_loss: 2.0841 - val_acc: 0.6472
Epoch 6/10
1063/1063 [==============================] - 157s 148ms/step - loss: 1.5161 - acc: 0.6675 - val_loss: 2.1378 - val_acc: 0.6444
Epoch 7/10
1063/1063 [==============================] - 157s 148ms/step - loss: 1.3373 - acc: 0.6949 - val_loss: 2.1974 - val_acc: 0.6439
Epoch 8/10
1063/1063 [============

In [36]:
model.save('chatbot.h5')
model.save_weights('chatbot_weights.h5')

In [37]:
enc_model = tf.keras.models.Model(enc_inp, [encoder_outputs, enc_states])


decoder_state_input_h = tf.keras.layers.Input(shape=( 400 * 2,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 400 * 2,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]

#decoder_output = dec_dense(decoder_outputs)

dec_model = tf.keras.models.Model([dec_inp, decoder_states_inputs],
                                      [decoder_outputs] + decoder_states)

In [39]:
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    
    prepro1 = input("you : ")
    prepro = [prepro1]
    
    try:
        txt = []
        for x in prepro:
            lst = []
            for y in x.split():
                lst.append(vocab[y])
            txt.append(lst)
        txt = pad_sequences(txt, 20, padding='post')


        ###
        enc_op, stat = enc_model.predict( txt ) 

        empty_target_seq = np.zeros( ( 1 , 1) )
        empty_target_seq[0, 0] = vocab['<SOS>']
        stop_condition = False
        decoded_translation = ''


        while not stop_condition :

            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + stat )

            ###
            ###########################
            attn_op, attn_state = attn_layer([enc_op, dec_outputs])
            decoder_concat_input = Concatenate(axis=-1)([dec_outputs, attn_op])
            decoder_concat_input = dec_dense(decoder_concat_input)
            ###########################

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )

            sampled_word = inv_vocab[sampled_word_index] + ' '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word           


            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            stat = [ h , c ] 
    except:
        pass

    print("chatbot attention : ", decoded_translation )
    print("==============================================")

##########################################
#       start chatting ver. 1.0          #
##########################################
you : hey
chatbot attention :  hey 
you : i love you
chatbot attention :  i love you 
you : how are you
chatbot attention :  insane i am just wondering how much how do you feel about 
you : thanks
chatbot attention :  you are welcome 
you : are you back at party
chatbot attention :  no 
you : when will you back
chatbot attention :  i will be in the morning 
you : I haven't seen you for two weeks
chatbot attention :  i will be in the morning 
you : what's that make you
chatbot attention :  i will be in the morning 
you : i hate you
chatbot attention :  i know 
you : how do you knwo
chatbot attention :  i know 
you : how?
chatbot attention :  i know 
you : ok..you are boring...bye
chatbot attention :  i know 
you : bye
chatbot attention :  bye 
you : thanks
chatbot attention :  you are welcome 
you : hey
chatbot attention :  hey 
you : how are you/
chatbot atte